In [1]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from tqdm import tqdm
import requests
from lxml.html import fromstring
from csv import writer

tqdm.pandas()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [8]:
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.proxy import Proxy, ProxyType

In [2]:
def driver_chrome(link, PROXY):
    
    options = webdriver.ChromeOptions()
    
    options.add_argument('headless')
    options.add_argument('window-size=1200x600')    
    options.add_argument('--proxy-server=%s' % PROXY)
    driver = webdriver.Chrome(executable_path="1/chromedriver_1", options=options)
    driver.get(link)
    return driver

In [3]:
def element(driver, XPATH):
    try:    
        element = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, XPATH)))
        return element.text
    except TimeoutException as TE:
        return None

In [93]:
def get_mail(driver):
        
    result_1 = element(driver, '//*[@id="email-finder-form"]/div[2]/div[1]/span')
    result_2 = element(driver, '//*[@id="email-finder-form"]/div[2]/div[2]/div[1]/div/div[2]')
    result_404 = element(driver, '/html/body/div/div')
    
    if result_404 is not None and result_404 == '404':
        return 404
    
    if result_1 is not None:
        return result_1
    
    if result_2 is not None:
        return result_2
    
    else:
        return None

In [119]:
# def get_proxy_list():
#     r = requests.get("https://www.us-proxy.org/")
# #     r = requests.get("https://free-proxy-list.net/anonymous-proxy.html")
#     soup = BeautifulSoup(r.text)
#     table = soup.find_all("table")[0]
#     df = pd.read_html(str(table))[0]
#     df = df[df["Https"]=="no"]
#     df["Port"] = df["Port"].astype(int).astype(str) 
#     proxies_list = list((df["IP Address"] +":"+ df["Port"]).values)
#     return proxies_list

# def get_proxy_list():
#     url = 'https://free-proxy-list.net/'
#     response = requests.get(url)
#     parser = fromstring(response.text)
#     proxies = set()
#     for i in parser.xpath('//tbody/tr')[:10]:
#         if i.xpath('.//td[7][contains(text(),"yes")]'):
#             #Grabbing IP and corresponding PORT
#             proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
#             proxies.add(proxy)
#     return list(proxies)

In [120]:
def get_proxy_address(proxies):
    if len(proxies) > 1:
        proxy = proxies[0]
        del proxies[0]
        return proxy
    else:
        return None

In [121]:
def get_proxy(proxies):
    while True:
        print(len(proxies))
        PROXY = get_proxy_address(proxies)
        if PROXY:
            return PROXY, proxies
            break
        else:
            print("NEW PROXY")
            proxies = get_proxy_list()[:10]
    

In [122]:
def check_acc(link):
    if os.path.exists("accounts.csv"):
        accounts = pd.read_csv("accounts.csv",header=None)
        accounts.columns = ["Full Name", "email", 'website', 'link']
        if link not in accounts.link.values:
            return True

In [4]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [124]:
def get_result(row, PROXY, proxies):    
    link = row["link"]
    website = row["website"]
    name = row["name"]    
    if link is not None and check_acc(link):
        while True:
            print(PROXY, link)
            driver = driver_chrome(link, PROXY)
            email = get_mail(driver)        
            print("EMAIL:",email)
            if email is None or email == "You have reached your free quota. Please create a free account to search more email addresses.":
                print("TimeoutException")
                PROXY, proxies = get_proxy(proxies)            
                driver.close() 
            else:
                print("EMAIL:",email)
                driver.close()
                append_list_as_row('accounts.csv', [name, email, website, link])
                return email
    else:
        return None

In [127]:
proxies = get_proxy_list()
PROXY, proxies = get_proxy(proxies)

10


In [128]:
df["results"] = df.progress_apply(lambda x: get_result(x, PROXY, proxies), axis=1)

  0%|          | 0/21 [00:00<?, ?it/s]

194.116.162.188:3128 https://hunter.io/find/austininformatics.co.uk/Austin%20Informatics
EMAIL: You have reached your free quota. Please create a free account to search more email addresses.
TimeoutException
9
41.190.92.99:43871 https://hunter.io/find/austininformatics.co.uk/Austin%20Informatics
EMAIL: None
TimeoutException
8
144.76.214.157:1080 https://hunter.io/find/austininformatics.co.uk/Austin%20Informatics
EMAIL: None
TimeoutException
7
181.101.164.36:7071 https://hunter.io/find/austininformatics.co.uk/Austin%20Informatics


 81%|████████  | 17/21 [10:26<02:27, 36.88s/it]


TimeoutException: Message: timeout: Timed out receiving message from renderer: -0.000
  (Session info: headless chrome=81.0.4044.129)


In [30]:
df.loc[5]

name                                       Bahador Zabihiyan
website                            https://zabihiyan.com/en/
link       https://hunter.io/find/zabihiyan.com/en/Bahado...
Name: 5, dtype: object

In [ ]:
https://hunter.io/find/zabihiyan.com/Bahador%20Zabihiyan

In [109]:
# pd.read_csv("accounts.csv",header=None)
# .drop_duplicates(subset=[0]).to_csv("accounts.csv", index=None)

In [ ]:
# def get_proxy_list():
#     r = requests.get("https://www.us-proxy.org/")
# #     r = requests.get("https://free-proxy-list.net/anonymous-proxy.html")
#     soup = BeautifulSoup(r.text)
#     table = soup.find_all("table")[0]
#     df = pd.read_html(str(table))[0]
#     df = df[df["Https"]=="no"]
#     df["Port"] = df["Port"].astype(int).astype(str) 
#     proxies_list = list((df["IP Address"] +":"+ df["Port"]).values)
#     return proxies_list

def get_proxy_list():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return list(proxies)